# **Heart Failure Dataset**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation: CROSS VALIDATION
from sklearn.model_selection import cross_val_predict #prediction

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, f1_score, recall_score, precision_score, accuracy_score, classification_report


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
cat_cols=[c for c in data.columns if data[c].isin([0,1]).all()]
cat_cols.remove('DEATH_EVENT')
print(cat_cols)

In [ ]:
num_cols=[c for c in data.columns if c not in cat_cols]
num_cols.remove('DEATH_EVENT')
num_cols

## CATEGORICAL FEATURES

In [ ]:
c=pd.melt(data, id_vars='DEATH_EVENT', value_vars=cat_cols)
grid=sns.FacetGrid(c, col_wrap=3, col='variable', sharex=False, sharey=True, height=4)
grid.map(sns.barplot, 'value', 'DEATH_EVENT', palette='Set2', order=[0,1])

In [ ]:
c=pd.melt(data, id_vars='DEATH_EVENT', value_vars=cat_cols)
sns.catplot(x='value', hue='DEATH_EVENT', col='variable', data=c, palette='RdYlBu', kind="count", 
            col_wrap=3, height=4, aspect=.7, sharex=False, sharey=True)


## NUMERICAL FEATURES

In [ ]:
data[num_cols]

In [ ]:
melt=pd.melt(data, value_vars=num_cols)
grid=sns.FacetGrid(melt, col='variable',col_wrap=3, sharex=False, sharey=False, height=4, aspect=1)
grid.map(sns.distplot, 'value')

In [ ]:
melt=pd.melt(data, value_vars=num_cols)
grid=sns.FacetGrid(melt, col='variable',col_wrap=2, sharex=False, sharey=False, aspect=2)
grid.map(sns.boxplot, 'value')

## Target Variable: DEATH_EVENT

In [ ]:
sns.countplot(data['DEATH_EVENT'])

We are dealing with imbalanced classification problem.

## CORRELATIONS

In [ ]:
correlation=data.corr().abs()
fig=plt.figure(figsize=(15,12))
ax=plt.gca()
sns.heatmap(correlation, annot=True, cmap='RdYlBu', mask=np.triu(correlation, k=0), ax=ax)

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
columns=correlation.nlargest(5, 'DEATH_EVENT').index.tolist()
correlation.nlargest(5, 'DEATH_EVENT')[columns].style.background_gradient(cmap=cm)

* time(?)
* serum_creatinine    
* ejection_fraction 
are the top 3 correlated vars with our target.

# MODELLING

In [ ]:
X=data[columns].drop('DEATH_EVENT', axis=1)
y=data['DEATH_EVENT']
print(columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2, shuffle=True)

In [ ]:
# Feature Scaling

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Base Model: Logistic Regression

In [ ]:
model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)
log_preds = model.predict(X_test)
print('Logistic Regression ', accuracy_score(y_test, log_preds))

In [ ]:
plot_confusion_matrix(model, X_test, y_test) 

In [ ]:
print('Logistic Regression f1-score:', f1_score(y_test, log_preds))
print('Logistic Regression precision:', precision_score(y_test, log_preds))
print('Logistic Regression recall:', recall_score(y_test, log_preds))

In [ ]:
print(classification_report(y_test, log_preds))

**Cross Validation**

In [ ]:
sklearn.metrics.SCORERS.keys()

In [ ]:
#Stratified Kfold for imbalanced classification problem.
stkfold=StratifiedKFold(n_splits=5)
scores=cross_val_score(model, X, y, cv = stkfold, scoring = "f1")
scores.mean()

## K Nearest Neighbor

In [ ]:
knn_scores=[]
for i in range(1,25):
    model=KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    knn_preds=model.predict(X_test)
    knn_scores.append(f1_score(y_test, knn_preds))

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(range(1,25,1), knn_scores)
ticks=plt.gca().set_xticks(range(1,25))

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_train, y_train)
knn_preds=model.predict(X_test)
print("f1 score:",f1_score(y_test, knn_preds))
print()
print(classification_report(y_test, knn_preds))

## Random Forest

In [ ]:
rf_model=RandomForestClassifier(max_leaf_nodes=100, max_depth=10,random_state=0)
rf_model.fit(X_train, y_train)
rf_preds=rf_model.predict(X_test)
print("f1 score:",f1_score(y_test, rf_preds))
print()
print(classification_report(y_test, rf_preds))